## Search algorithms within Optuna



We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

# Data Pre-processing

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.


In [2]:
data = pd.read_csv('balanced_bank.csv', index_col = 0)
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
34579,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,...,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
446,42,technician,married,professional.course,no,no,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28870,45,blue-collar,married,unknown,no,yes,no,cellular,apr,thu,...,1,999,1,failure,-1.8,93.075,-47.1,1.410,5099.1,yes
30452,60,admin.,married,university.degree,no,yes,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.354,5099.1,yes
28937,38,blue-collar,married,basic.9y,no,yes,no,cellular,apr,fri,...,1,999,1,failure,-1.8,93.075,-47.1,1.405,5099.1,no
16739,41,technician,single,university.degree,no,no,no,cellular,jul,thu,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9280 entries, 34579 to 16807
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             9280 non-null   int64  
 1   job             9280 non-null   object 
 2   marital         9280 non-null   object 
 3   education       9280 non-null   object 
 4   default         9280 non-null   object 
 5   housing         9280 non-null   object 
 6   loan            9280 non-null   object 
 7   contact         9280 non-null   object 
 8   month           9280 non-null   object 
 9   day_of_week     9280 non-null   object 
 10  duration        9280 non-null   int64  
 11  campaign        9280 non-null   int64  
 12  pdays           9280 non-null   int64  
 13  previous        9280 non-null   int64  
 14  poutcome        9280 non-null   object 
 15  emp.var.rate    9280 non-null   float64
 16  cons.price.idx  9280 non-null   float64
 17  cons.conf.idx   9280 non-nul

In [4]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
34579,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,...,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
446,42,technician,married,professional.course,no,no,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


In [5]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [6]:
data.loan.value_counts()

no         7691
yes        1371
unknown     218
Name: loan, dtype: int64

In [7]:
data.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

# Categorical Encoding

In [8]:
categorical_cols = ['job','marital','education','month','day_of_week'] 
data_encoded = pd.get_dummies(data, drop_first=True,columns = categorical_cols)

# Binary encoding


In [9]:
binary_cat_cols = ['contact','housing','loan','poutcome','default']

for i in binary_cat_cols:
    if len(data_encoded[i].unique())==2:
        data_encoded[i].replace({data_encoded[i].unique()[0]: 0, data_encoded[i].unique()[1]: 1, 'unknown':-1}, inplace=True)

    else:
        data_encoded[i].replace({data_encoded[i].unique()[0]: 0, data_encoded[i].unique()[1]: 1, data_encoded[i].unique()[2]: 2, 'unknown':-1}, inplace=True)


# Label encoding

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data_encoded['y'] = label_encoder.fit_transform(data_encoded['y'])

In [11]:
data_encoded.dtypes

age                                int64
default                            int64
housing                            int64
loan                               int64
contact                            int64
duration                           int64
campaign                           int64
pdays                              int64
previous                           int64
poutcome                           int64
emp.var.rate                     float64
cons.price.idx                   float64
cons.conf.idx                    float64
euribor3m                        float64
nr.employed                      float64
y                                  int32
job_blue-collar                    uint8
job_entrepreneur                   uint8
job_housemaid                      uint8
job_management                     uint8
job_retired                        uint8
job_self-employed                  uint8
job_services                       uint8
job_student                        uint8
job_technician  

# Scaling

In [12]:
print(data_encoded.shape)

X = data_encoded.loc[:,data_encoded.columns!='y']
print(X.shape)

y = data_encoded.loc[:,'y']
print(y.shape)

(9280, 50)
(9280, 49)
(9280,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [14]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Baseline model

In [15]:
model = xgb.XGBClassifier(nthread=-1)  
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
accuracy

0.8854064642507345

# HyperParameter optimization with Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [17]:
def objective(trial):
    
    
    param = {
        "objective": 'binary:logistic',
        'reg_lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3,1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [i/10.0 for i in range(4,11)]),
        'subsample': trial.suggest_categorical('subsample', [i/10.0 for i in range(4,11)]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.300000012]),
        'n_estimators': trial.suggest_int('n_estimators',100,500),
        'max_depth': trial.suggest_categorical('max_depth', [5,6,7,9,11,13,15,17,20]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param, nthread=-1)  
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [18]:
random_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)

tpe_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)

cmaes_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

cmaes_study.optimize(objective, n_trials=100)
tpe_study.optimize(objective, n_trials=100)
random_study.optimize(objective, n_trials=100)


[I 2021-12-15 14:20:31,616] A new study created in memory with name: no-name-e6a918c6-54ae-4b43-8177-65ab0a3f90f4
[I 2021-12-15 14:20:31,631] A new study created in memory with name: no-name-e12436fe-5f8c-4faa-9499-67bb083d2f3b
[I 2021-12-15 14:20:31,634] A new study created in memory with name: no-name-2d39f5ef-7326-4545-872b-e6dfe4683366
[I 2021-12-15 14:20:33,695] Trial 0 finished with value: 0.871041462618348 and parameters: {'lambda': 0.0024397246556234964, 'alpha': 0.007382954871191506, 'gamma': 0.04316971959666753, 'colsample_bytree': 0.4, 'subsample': 0.9, 'learning_rate': 0.014, 'n_estimators': 395, 'max_depth': 11, 'min_child_weight': 213}. Best is trial 0 with value: 0.871041462618348.
[W 2021-12-15 14:20:33,736] The parameter 'colsample_bytree' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress th

[W 2021-12-15 14:20:39,232] The parameter 'learning_rate' in trial#4 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:20:39,232] The parameter 'max_depth' in trial#4 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-12-15 14:20:40,961] Trial 4 finished with value: 0.8566764609859615 and parameters: {'lambda': 0.17565209709760063

[W 2021-12-15 14:20:46,323] The parameter 'subsample' in trial#8 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:20:46,331] The parameter 'learning_rate' in trial#8 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:20:46,333] The parameter 'max_depth' in trial#8 is sampled independently by using `RandomSampler` instead

[W 2021-12-15 14:20:53,529] The parameter 'colsample_bytree' in trial#12 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:20:53,529] The parameter 'subsample' in trial#12 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:20:53,529] The parameter 'learning_rate' in trial#12 is sampled independently by using `RandomSample

[I 2021-12-15 14:21:01,118] Trial 15 finished with value: 0.8677766895200784 and parameters: {'lambda': 0.08237934266337686, 'alpha': 0.06922198290147676, 'gamma': 0.018010658245755644, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.012, 'n_estimators': 299, 'max_depth': 13, 'min_child_weight': 149}. Best is trial 13 with value: 0.8756121449559255.
[W 2021-12-15 14:21:01,128] The parameter 'colsample_bytree' in trial#16 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[W 2021-12-15 14:21:01,128] The parameter 'subsample' in trial#16 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `Cm

...

[I 2021-12-15 14:30:31,326] Trial 7 finished with value: 0.8723473718576559 and parameters: {'lambda': 0.00232654229083491, 'alpha': 0.38891259290068325, 'gamma': 0.02234210197214249, 'colsample_bytree': 0.8, 'subsample': 1.0, 'learning_rate': 0.300000012, 'n_estimators': 349, 'max_depth': 9, 'min_child_weight': 278}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:30:32,965] Trial 8 finished with value: 0.8579823702252694 and parameters: {'lambda': 1.5978550007679164, 'alpha': 0.0016078971817456464, 'gamma': 0.06100189351340322, 'colsample_bytree': 0.6, 'subsample': 0.9, 'learning_rate': 0.009, 'n_estimators': 252, 'max_depth': 15, 'min_child_weight': 234}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:30:34,534] Trial 9 finished with value: 0.881815213842638 and parameters: {'lambda': 5.895018672564375, 'alpha': 3.0289027575550445, 'gamma': 0.17204797748012074, 'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 0.300000012, 'n_estimators':

[I 2021-12-15 14:31:14,740] Trial 29 finished with value: 0.8788769180541952 and parameters: {'lambda': 1.7519986648029857, 'alpha': 0.007436283645776847, 'gamma': 0.17698218241463312, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.300000012, 'n_estimators': 239, 'max_depth': 6, 'min_child_weight': 84}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:31:16,511] Trial 30 finished with value: 0.85145282402873 and parameters: {'lambda': 0.038289437290317516, 'alpha': 0.0017650391031599217, 'gamma': 0.04109597762289261, 'colsample_bytree': 0.8, 'subsample': 0.9, 'learning_rate': 0.008, 'n_estimators': 206, 'max_depth': 15, 'min_child_weight': 173}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:31:17,817] Trial 31 finished with value: 0.8566764609859615 and parameters: {'lambda': 0.6498845698545593, 'alpha': 0.0014134451066722944, 'gamma': 0.04312759023735436, 'colsample_bytree': 1.0, 'subsample': 0.9, 'learning_rate': 0.018, 'n_estimators'

[I 2021-12-15 14:32:03,786] Trial 51 finished with value: 0.8743062357166177 and parameters: {'lambda': 0.006694720550492909, 'alpha': 0.03989989852880341, 'gamma': 0.0051067831679929825, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.018, 'n_estimators': 372, 'max_depth': 5, 'min_child_weight': 102}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:32:06,248] Trial 52 finished with value: 0.8560235063663075 and parameters: {'lambda': 0.0025030496347307287, 'alpha': 0.1606713467891515, 'gamma': 0.15071539070444087, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.009, 'n_estimators': 409, 'max_depth': 7, 'min_child_weight': 124}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:32:09,380] Trial 53 finished with value: 0.8566764609859615 and parameters: {'lambda': 0.0054164727590674125, 'alpha': 0.7776632067001685, 'gamma': 0.05167555390844606, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.008, 'n_estimators': 4

[I 2021-12-15 14:32:50,565] Trial 73 finished with value: 0.8589618021547503 and parameters: {'lambda': 0.01488942270301306, 'alpha': 0.005884173662608366, 'gamma': 0.0021581923210066685, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.014, 'n_estimators': 128, 'max_depth': 5, 'min_child_weight': 72}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:32:52,285] Trial 74 finished with value: 0.8671237349004244 and parameters: {'lambda': 0.13771683296867934, 'alpha': 0.01950647248912753, 'gamma': 0.011078580356736093, 'colsample_bytree': 0.4, 'subsample': 0.7, 'learning_rate': 0.008, 'n_estimators': 242, 'max_depth': 11, 'min_child_weight': 104}. Best is trial 6 with value: 0.8847535096310807.
[I 2021-12-15 14:32:53,243] Trial 75 finished with value: 0.8775710088148874 and parameters: {'lambda': 0.38252917829620264, 'alpha': 0.01126131332832667, 'gamma': 0.0014039906865912467, 'colsample_bytree': 0.4, 'subsample': 0.9, 'learning_rate': 0.300000012, 'n_estimat

[I 2021-12-15 14:33:31,986] Trial 95 finished with value: 0.8694090760692131 and parameters: {'lambda': 0.002060028077120148, 'alpha': 0.011653499485161797, 'gamma': 0.0045826618290063246, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.01, 'n_estimators': 375, 'max_depth': 17, 'min_child_weight': 132}. Best is trial 89 with value: 0.8863858961802155.
[I 2021-12-15 14:33:36,569] Trial 96 finished with value: 0.8841005550114267 and parameters: {'lambda': 9.149824562015786, 'alpha': 0.022412585387022708, 'gamma': 0.07582914163806408, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.008, 'n_estimators': 349, 'max_depth': 17, 'min_child_weight': 2}. Best is trial 89 with value: 0.8863858961802155.
[I 2021-12-15 14:33:39,245] Trial 97 finished with value: 0.8553705517466537 and parameters: {'lambda': 0.055734943294242646, 'alpha': 0.013427662898145263, 'gamma': 0.0020512225235654598, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.01, 'n_estimators':

In [19]:
search_space = {
    "n_estimators": [100, 500, 1000],
    "max_depth": [5,6,7,9,11,13,15,17,20],
    "min_samples_split": [0.1, 1.0],
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],

}

def objective(trial):
    
    
    param = {
        'n_estimators': trial.suggest_int('n_estimators',100,500),
        'max_depth': trial.suggest_categorical('max_depth', [5,6,7,9,11,13,15,17,20]),
    }
    model = xgb.XGBClassifier(**param)  
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [20]:
grid_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

grid_study.optimize(objective)

[I 2021-12-15 14:53:33,439] A new study created in memory with name: no-name-a7e77fbd-fb3b-4d59-a434-4c09c56dda64
[I 2021-12-15 14:53:35,891] Trial 0 finished with value: 0.8844270323212536 and parameters: {'n_estimators': 100, 'max_depth': 6}. Best is trial 0 with value: 0.8844270323212536.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:54:25,143] Trial 1 finished with value: 0.8795298726738492 and parameters: {'n_estimators': 1000, 'max_depth': 15}. Best is trial 0 with value: 0.8844270323212536.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 15:01:54,381] Trial 29 finished with value: 0.880835781913157 and parameters: {'n_estimators': 1000, 'max_depth': 13}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:02:00,473] Trial 30 finished with value: 0.8798563499836761 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:02:06,631] Trial 31 finished with value: 0.8798563499836761 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:02:08,891] Trial 32 finished with value: 0.8850799869409076 and parameters: {'n_estimat

[I 2021-12-15 15:11:47,308] Trial 49 finished with value: 0.8782239634345413 and parameters: {'n_estimators': 1000, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 15:12:35,678] Trial 50 finished with value: 0.880835781913157 and parameters: {'n_estimators': 1000, 'max_depth': 13}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:12:39,515] Trial 51 finished with value: 0.8798563499836761 and parameters: {'n_estimators': 100, 'max_depth': 9}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:12:42,116] Trial 52 finished with value: 0.8844270323212536 and parameters: {'n_estimat

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 15:20:35,264] Trial 77 finished with value: 0.8801828272935031 and parameters: {'n_estimators': 1000, 'max_depth': 7}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:20:38,363] Trial 78 finished with value: 0.8844270323212536 and parameters: {'n_estimators': 100, 'max_depth': 6}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:20:46,091] Trial 79 finished with value: 0.8798563499836761 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:20:53,101] Trial 80 finished with value: 0.881488736532811 and parameters: {'n_estimator

[I 2021-12-15 15:31:55,748] Trial 103 finished with value: 0.8778974861247143 and parameters: {'n_estimators': 500, 'max_depth': 20}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:32:04,625] Trial 104 finished with value: 0.881488736532811 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 15:32:08,436] Trial 105 finished with value: 0.8834476003917727 and parameters: {'n_estimators': 100, 'max_depth': 7}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 15:33:09,491] Trial 106 finished with value: 0.8782239634345413 and parameters: {'n_estim

...

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:12:07,302] Trial 339 finished with value: 0.8765915768854065 and parameters: {'n_estimators': 1000, 'max_depth': 6}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:12:31,084] Trial 340 finished with value: 0.8765915768854065 and parameters: {'n_estimators': 1

[I 2021-12-15 17:20:59,690] Trial 363 finished with value: 0.8782239634345413 and parameters: {'n_estimators': 1000, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:21:51,423] Trial 364 finished with value: 0.880835781913157 and parameters: {'n_estimators': 1000, 'max_depth': 13}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The va

[I 2021-12-15 17:31:45,164] Trial 388 finished with value: 0.8765915768854065 and parameters: {'n_estimators': 1000, 'max_depth': 6}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 17:31:49,125] Trial 389 finished with value: 0.8834476003917727 and parameters: {'n_estimators': 100, 'max_depth': 7}. Best is trial 4 with value: 0.8850799869409076.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:33:11,690] Trial 390 finished with value: 0.8801828272935031 and parameters: {'n_estimators': 1000, 'max_depth': 9}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 17:34:22,327] Trial 391 finished with value: 0.8778974861247143 and parameters: {'n_esti

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:57:54,444] Trial 418 finished with value: 0.8805093046033301 and parameters: {'n_estimators': 1000, 'max_depth': 20}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 17:58:34,629] Trial 419 finished with value: 0.8805093046033301 and parameters: {'n_estimators': 500, 'max_depth': 9}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 17:59:37,337] Trial 420 finished with value: 0.8785504407443683 and parameters: {'n_estimators': 500, 'max_depth': 17}. Best is trial 4 with value: 0.8850799869409076.
[I 2021-12-15 17:59:42,758] Trial 421 finished with value: 0.8850799869409076 and parameters: {'n_est

KeyboardInterrupt: 

In [21]:
tstamp1 = cmaes_study.trials_dataframe()['datetime_complete'].min()
tstamp2 = grid_study.trials_dataframe()['datetime_complete'].max()

In [22]:
temp = tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1
temp

Timedelta('0 days 03:39:09.064630')

In [24]:
print('GRID SEARCH: ',grid_study.best_value)
print('CMAES: ',cmaes_study.best_value)
print('TPE: ',tpe_study.best_value)
print('RANDOM SEARCH : ',random_study.best_value)
print('BASELINE MODEL: ',accuracy)

GRID SEARCH:  0.8850799869409076
CMAES:  0.8775710088148874
TPE:  0.8886712373490042
RANDOM SEARCH :  0.8896506692784851
BASELINE MODEL:  0.8854064642507345


In [1]:
# from sqlalchemy import create_engine
# from sqlalchemy.pool import StaticPool
# engine = create_engine('sqlite:///m.db',
#                     poolclass=StaticPool)

In [2]:
# study_name = 'example-l'  # Unique identifier of the study.
# study = optuna.create_study(study_name=study_name, storage='sqlite:///m.db', direction="maximize",
#     sampler=optuna.samplers.TPESampler(), load_if_exists=True
# )
# study.optimize(objective, n_trials=100)

In [41]:
tstamp1 = study.trials_dataframe()['datetime_complete'].min()
tstamp2 = study.trials_dataframe()['datetime_complete'].max()

tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1

Timedelta('0 days 00:10:26.128678')

In [47]:
study = optuna.load_study(study_name=study_name, storage='sqlite:///m.db',
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=200)


C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:27:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-12-15 01:27:48,772] Trial 220 finished with value: 0.82 and parameters: {'lambda': 0.009730066153068524, 'alpha': 0.0014918614135806128, 'gamma': 0.007692681137278601, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.018, 'n_estimators': 477, 'max_depth': 7, 'min_child_weight': 12}. Best is trial 199 with value: 0.91.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:27:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-12-15 01:27:51,475] Trial 221 finished with value: 0.88 and parameters: {'lambda': 0.02282858996099954, 'alpha': 0.0013505114662567812, 'gamma': 0.0017808609398968214, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.018, 'n_estimators': 500, 'max_depth': 7, 'min_child_weight': 1}. Best is trial 199 with value: 0.91.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[I 2021-12-15 01:34:13,909] Trial 417 finished with value: 0.81 and parameters: {'lambda': 0.01904414206789297, 'alpha': 0.001417293913532806, 'gamma': 0.00166433999722452, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.018, 'n_estimators': 468, 'max_depth': 13, 'min_child_weight': 15}. Best is trial 199 with value: 0.91.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:34:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-12-15 01:34:15,471] Trial 418 finished with value: 0.87 and parameters: {'lambda': 0.05076038847824833, 'alpha': 0.016063664030951632, 'gamma': 0.0023396125132332745, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.300000012, 'n_estimators': 462, 'max_depth': 20, 'min_child_weight': 6}. Best is trial 199 with value: 0.91.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:34:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-12-15 01:34:17,943] Trial 419 finished with value: 0.82 and parameters: {'lambda': 0.009533107800759453, 'alpha': 0.0012942438443647637, 'gamma': 0.004715182190083166, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.014, 'n_estimators': 491, 'max_depth': 13, 'min_child_weight': 11}. Best is trial 199 with value: 0.91.
